In [1]:
# -*- coding: utf-8 -*-
"""Outlook folder reader:
 Read specified folder and save items in the folder to csv

original:
https://www.laurivan.com/python-and-outlook-an-example/
see:
https://docs.python.jp/3/using/windows.html#pywin32
http://timgolden.me.uk/pywin32-docs/contents.html
http://timgolden.me.uk/pywin32-docs/html/com/win32com/HTML/QuickStartClientCom.html
http://fum125.hatenablog.com/entry/2016/11/12/233457
https://msdn.microsoft.com/ja-jp/VBA/Outlook-VBA/articles/recipient-type-property-outlook
https://docs.python.jp/3/library/csv.html?highlight=csv
https://msdn.microsoft.com/ja-jp/vba/outlook-vba/articles/propertyaccessor-object-outlook
"""
import win32com.client

DEBUG = False

# Outlook オブジェクト作成

In [2]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

# サポートメソッド

In [3]:
def items(outlook_items_object):
    array_size = outlook_items_object.Count
    for item_index in range(1,array_size+1):
        yield (item_index, outlook_items_object[item_index])
    return

        
def prop(outlook_object):
    return sorted( outlook_object._prop_map_get_.keys() )



# フォルダーのパース

In [4]:
roots = outlook.Folders

In [5]:
def parse_folder(folder, recursive=False, depth=0, index=1):
    if not folder:
        return
    print( " "*depth*2, u"(%i) [%s] [%s]" % (index, folder.Name, folder))
    if recursive:
        if folder.Folders:
            for index, folder in items(folder.Folders):
                parse_folder(folder, recursive, depth + 1, index)
    return



In [6]:
# コメントを外せば動作確認
#for index, folder in items(roots):
#    parse_folder(folder, recursive=True)

# 特定のフォルダのアイテム表示

In [7]:
def search(folders, query_path):
    if DEBUG:
        print(query_path)
    if not folders:
        return None
    if len(query_path)<=0:
        return None
    for index, folder in items(folders):
        if folder.Name == query_path[0]:
            if len(query_path)==1:
                return folder
            else:
                return search(folder.Folders, query_path[1:])
    return None


def get_mail_item_header(mail_item):
    prop_name = "http://schemas.microsoft.com/mapi/proptag/0x007D001E"
    return mail_item.PropertyAccessor.GetProperty(prop_name)


def arrange_item(mail_item, body=False, header=False):
    mail_from = mail_item.SenderEmailAddress + ";"
    mail_to = ""
    mail_cc = ""
    for i, recipient in items(mail_item.Recipients):
        if recipient.Type == win32com.client.constants.olTo:
            mail_to += recipient.Address + ";"
        if (recipient.Type == win32com.client.constants.olCC
            or recipient.Type == win32com.client.constants.olBCC):
            mail_cc += recipient.Address + ";"
        if recipient.Type == win32com.client.constants.olOriginator:
            mail_from += recipient.Address + ";"
    mail_subj = mail_item.Subject
    mail_sentdate = mail_item.SentOn
    mail_body = ""
    if body:
        mail_body = mail_item.Body
    mail_header = ""
    if header:
        mail_header = get_mail_item_header(mail_item)
    return [mail_from, mail_to, mail_cc, str(mail_sentdate), mail_subj
            , mail_body, mail_header]


def print_folderitems(folder, recursive=False):
    for index, item in items(folder.Items):
        print("\t".join([folder.Name] + arrange_item(item)))
    if recursive:
        for index, subfolder in items(folder.Folders):
            print_folderitems(subfolder, recursive)
    return



In [8]:
folder = search(roots, 'Outlook データ ファイル\\受信トレイ'.split('\\'))
print(folder.Name)
print_folderitems(folder, recursive=False)


受信トレイ


# 特定のフォルダのアイテムをCSV保存

In [9]:
import csv

def save_folderitems_sub(folder, csvwriter, recursive=False):
    for index, mail_item in items(folder.Items):
        item_arranged = arrange_item(mail_item, body=True, header=True)
        csvwriter.writerow([folder.Name] + item_arranged)
    if recursive:
        for index, folder in items(folder.Folders):
            save_folderitems_sub(folder, csvwriter, recursive)
    return

def save_folderitems(folder, save_filename, recursive=False):
    with open(save_filename, 'w', newline=''
              , encoding='utf-8', errors='replace') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["folder","from", "to", "cc", "sent date"
                         , "subject", "body", "internet mail header"])
        save_folderitems_sub(folder, writer, recursive)
    return


In [10]:
folder = search(roots, 'Outlook データ ファイル\\受信トレイ'.split('\\'))
print(folder.Name)
save_folderitems(folder, 'test.csv', recursive=False)

受信トレイ
